In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# decision trees

### Преимущества и недостатки деревьев решений
##### Преимущества:

-    **Интерпретируемость**: Одним из главных преимуществ деревьев решений является их легкость в интерпретации. Конечные пользователи могут легко понять, какие признаки были важны для предсказания.

-    **Гибкость**: Деревья решений могут использоваться как для классификации, так и для регрессии. Их можно адаптировать к различным типам задач.

-    **Отсутствие необходимости нормализации данных**: Деревья решений не требуют нормализации признаков, так как они работают только с разделениями данных.

-    **Устойчивость к выбросам**: Деревья решений менее чувствительны к выбросам, поскольку они концентрируются на разделениях, основанных на выборках, а не на отдельных значениях.

##### Недостатки:

-    **Переобучение**: Без применения ограничений (например, максимальной глубины) деревья могут сильно переобучаться, особенно на малых выборках.

-    **Нестабильность**: Небольшие изменения в данных могут привести к значительным изменениям структуры дерева, что делает модель менее устойчивой.

-    **Меньшая точность по сравнению с ансамблями**: Деревья решений по своей природе менее точны, чем более сложные методы, такие как случайные леса (Random Forests) или градиентный бустинг (Gradient Boosting).


### Переобучение в деревьях решений.
В чем оно проявляется?

-    Глубокие деревья: Дерево может создавать множество уровней, чтобы разделить каждую точку данных, даже если это вызвано шумом или случайными выбросами.
  
-    Чрезмерно детализированные разделения: Дерево может разделять данные до тех пор, пока не достигнет уровня, где каждая конечная группа (лист) будет содержать одно или несколько наблюдений. Это приводит к тому, что дерево подстраивается под мелкие особенности данных.


### 1. Ограничение глубины дерева
<div class='alert alert-box alert-success'>

```Ограничение глубины контролирует, насколько детализированным будет дерево. Меньшая глубина заставляет дерево принимать обобщённые решения и уменьшает вероятность переобучения.```
</div>

```from sklearn.tree import DecisionTreeClassifier```

##### Ограничиваем глубину дерева до 5 уровней
```tree = DecisionTreeClassifier(max_depth=5, random_state=42)```<br>
```tree.fit(X_train, y_train)```

### 2. Минимальное количество объектов в листе (min_samples_leaf)
<div class='alert alert-box alert-success'>

```Если конечный лист содержит слишком мало объектов, это может говорить о чрезмерной подстройке дерева под конкретные объекты. Увеличивая значение min_samples_leaf, мы требуем, чтобы дерево обрабатывало только более значимые группы объектов.```
</div>

```from sklearn.tree import DecisionTreeClassifier```

##### Минимум 10 объектов в каждом листе
```tree = DecisionTreeClassifier(min_samples_leaf=10, random_state=42)```<br>
```tree.fit(X_train, y_train)```

### 3. Минимальное количество объектов для разбиения (min_samples_split)
<div class='alert alert-box alert-success'>

```Когда количество объектов в узле становится слишком малым, дальнейшее разбиение может не принести пользы, а лишь усложнит модель. Ограничение минимального числа объектов для разбиения заставляет дерево принимать более обобщённые решения.```
</div>

```from sklearn.tree import DecisionTreeClassifier```

##### Минимум 20 объектов для разбиения узла
```tree = DecisionTreeClassifier(min_samples_split=20, random_state=42)```<br>
```tree.fit(X_train, y_train)```

### 4. Обрезка дерева (pruning)
<div class='alert alert-box alert-success'>

```Обрезка (pruning) — это процесс удаления узлов дерева, которые не дают значимого улучшения на новых данных. Этот метод применяется после построения дерева, чтобы удалить ненужные ветви, которые могут быть результатом подгонки под обучающую выборку.```
</div>
Существует два вида обрезки:

-    **Предварительная обрезка (Pre-pruning)**: Разбиения ограничиваются до того, как дерево будет построено полностью (задаётся через параметры max_depth, min_samples_leaf и т.д.).
-    **Пост-обрезка (Post-pruning)**: Обрезка выполняется после построения дерева. В sklearn это реализовано через параметр ccp_alpha, который задаёт величину регуляризации. Узлы, которые дают минимальное улучшение, удаляются, упрощая модель. Этот параметр задаёт силу регуляризации — чем выше значение, тем больше узлов будет обрезано.


```from sklearn.tree import DecisionTreeClassifier```<br>
```from sklearn.model_selection import GridSearchCV```

##### Настраиваем параметр ccp_alpha для пост-обрезки
```param_grid = {'ccp_alpha': [0.001, 0.01, 0.1, 0.5]}```<br>
```tree = DecisionTreeClassifier(random_state=42)```<br>
```grid_search = GridSearchCV(tree, param_grid, cv=5)```<br>
```grid_search.fit(X_train, y_train)```<br>
```best_tree = grid_search.best_estimator_```

In [5]:
iris = sns.load_dataset('iris')
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
iris.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [7]:
X = iris.drop(columns = 'species')
y = iris['species']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Decision tree without params

In [10]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [12]:
y_predict = tree.predict(X_test)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Decision tree with param_grid

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [5, 10 ,20]}

tree = DecisionTreeClassifier(random_state = 42)
grid_search = GridSearchCV(tree, param_grid=param_grid, cv = 5)

In [16]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [5, 10, 20]})

In [17]:
grid_search.best_params_

{'max_depth': 10}

In [19]:
y_predict = grid_search.best_estimator_.predict(X_test)

In [20]:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [22]:
y_predict == y_test

73     True
18     True
118    True
78     True
76     True
31     True
64     True
141    True
68     True
82     True
110    True
12     True
36     True
9      True
19     True
56     True
104    True
69     True
55     True
132    True
29     True
127    True
26     True
128    True
131    True
145    True
108    True
143    True
45     True
30     True
Name: species, dtype: bool

In [23]:
tree = DecisionTreeClassifier(random_state = 42, max_depth=3)

In [24]:
tree.fit(X_train, y_train)
y_predict = tree.predict(X_test)

In [25]:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [26]:
y_test == y_predict

73     True
18     True
118    True
78     True
76     True
31     True
64     True
141    True
68     True
82     True
110    True
12     True
36     True
9      True
19     True
56     True
104    True
69     True
55     True
132    True
29     True
127    True
26     True
128    True
131    True
145    True
108    True
143    True
45     True
30     True
Name: species, dtype: bool

In [37]:
from sklearn.model_selection import GridSearchCV
param_grid = (
                {
                 # 'max_depth': [5, 10 ,20], 
                 # 'min_samples_leaf': [20, 40, 60],
                 # 'min_samples_split': [30, 40, 50],
                 'ccp_alpha': [0.001, 0.01, 0.1]
                 }
)

tree = DecisionTreeClassifier(random_state = 42)
grid_search = GridSearchCV(tree, param_grid=param_grid, cv = 5)
grid_search.fit(X_train, y_train)
y_predict = grid_search.best_estimator_.predict(X_test)
print(grid_search.best_params_)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

{'ccp_alpha': 0.001}
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

